In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-03-01 02:49:52.307509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 02:49:52.307649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 02:49:52.478572: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from sklearn.neural_network import MLPClassifier

In [3]:
df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
df.head()

,uniqueID,classLabel,classID,source,imageFile
0,2,occluded-specimens,8,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,6,L,79599db2ac9092b6.jpg
2,4,illustrations-color,2,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,2,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,0,AK,041a1c6e73313638.jpg


In [4]:
df['classLabel'].value_counts()

classLabel
live-plants                       8200
illustrations-color               8196
illustrations-gray                8196
biocultural-specimens             8195
micrographs-transmission-light    8195
occluded-specimens                8194
animal-specimens                  8194
mixed-pressed-specimens           8193
microscope-slides                 8192
ordinary-pressed-specimens        8191
Name: count, dtype: int64

In [5]:
df['classID'].value_counts()

classID
4    8200
2    8196
3    8196
1    8195
5    8195
8    8194
0    8194
7    8193
6    8192
9    8191
Name: count, dtype: int64

In [6]:
df['source'].value_counts()

source
BHL      12691
L         8812
Tw        5274
K         5261
NY        5183
P         4755
BR        3789
Met       3438
F         3365
US        3064
E         2977
AK        2307
MICH      2163
CAS       1825
MO        1725
GH        1573
C         1564
YU        1380
O         1294
RSA       1037
KY         655
UA         655
FMNH       654
TU         647
MZH        612
MPU        603
NMR        600
MCZ        565
YPM        565
ASU        538
MA         507
CHNDM      476
UMMZ       452
SDNHM      337
NHMD       282
TTU        260
UHIM        61
Name: count, dtype: int64

In [7]:
df.shape

(81946, 5)

In [8]:
np.any(pd.isna(df))

False

## Cleaning the Data

In [9]:
X_train = df.drop(columns=['classID'])
y_train = df['classID']
X_train.head()

,uniqueID,classLabel,source,imageFile
0,2,occluded-specimens,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,L,79599db2ac9092b6.jpg
2,4,illustrations-color,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,AK,041a1c6e73313638.jpg


In [10]:
y_train.head()

0    8
1    6
2    2
3    2
4    0
Name: classID, dtype: int64

In [11]:
df.columns

Index(['uniqueID', 'classLabel', 'classID', 'source', 'imageFile'], dtype='object')

In [12]:
X_cols = X_train.columns
X_cols

Index(['uniqueID', 'classLabel', 'source', 'imageFile'], dtype='object')

In [13]:
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

# Load your data table containing image paths and additional information
# data_table = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train')
path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train/'

# Function to load and preprocess images
def load_and_preprocess_images(df):
    images = []
    all_img_info = []
    labels = []
    for index, row in df.iterrows():
        img_path = path + row['imageFile']  # Replace 'image_path_column' with the column name containing image paths
        label = row['classID']  # Replace 'label_column' with the column name containing labels
        curr_img_info = row[X_cols].values  # Replace 'additional_column1', 'additional_column2', etc. with the columns containing additional information
        curr_img = cv2.imread(img_path)
        curr_img = cv2.resize(curr_img, (32, 32))  # Resize the image if necessary
        curr_img = curr_img / 255.0  # Normalize the pixel values
        images.append(curr_img)
        all_img_info.append(curr_img_info)
        labels.append(label)
    return np.array(images), np.array(all_img_info), np.array(labels)

In [14]:
# Load and preprocess images and itionalitional information
images, all_img_info, labels = load_and_preprocess_images(df)

In [15]:
len(images)

81946

In [16]:
len(all_img_info)

81946

In [17]:
len(labels)

81946

In [18]:
# Define the combined input
image_input = Input(shape=(32, 32, 3))
additional_input = Input(shape=(len(X_cols)))  # Replace num_additional_columns with the number of additional columns

## Creating the model

In [19]:
from tensorflow.keras.metrics import AUC

# CNN for image processing
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Flatten()(x)

# Dense layers
x = Dense(1024, activation='relu')(x)

# Output layer
output = Dense(10, activation='softmax')(x)
labels = labels.astype('int32')

# Define the model
model = Model(inputs=image_input, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(images, labels, batch_size=32, epochs=6, validation_split=0.2)


Epoch 1/6
2049/2049 [==============================] - 71s 34ms/step - loss: 0.9007 - accuracy: 0.6809 - val_loss: 0.5697 - val_accuracy: 0.8024
Epoch 2/6
2049/2049 [==============================] - 68s 33ms/step - loss: 0.5283 - accuracy: 0.8139 - val_loss: 0.4694 - val_accuracy: 0.8350
Epoch 3/6
2049/2049 [==============================] - 67s 33ms/step - loss: 0.4443 - accuracy: 0.8427 - val_loss: 0.5092 - val_accuracy: 0.8214
Epoch 4/6
2049/2049 [==============================] - 67s 33ms/step - loss: 0.3841 - accuracy: 0.8645 - val_loss: 0.4665 - val_accuracy: 0.8412
Epoch 5/6
2049/2049 [==============================] - 68s 33ms/step - loss: 0.3370 - accuracy: 0.8798 - val_loss: 0.4175 - val_accuracy: 0.8525
Epoch 6/6
2049/2049 [==============================] - 67s 33ms/step - loss: 0.3055 - accuracy: 0.8906 - val_loss: 0.3686 - val_accuracy: 0.8757


## Making Predictions Using the Model

In [20]:
test_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test/'
def test_load_and_preprocess_images(df):
    images = []
    labels = []
    for index, row in df.iterrows():
        img_path = test_path + row['imageFile']  # Replace 'image_path_column' with the column name containing image paths
        label = row['uniqueID']  # Replace 'label_column' with the column name containing labels
        curr_img = cv2.imread(img_path)
        curr_img = cv2.resize(curr_img, (32, 32))  # Resize the image if necessary
        curr_img = curr_img / 255.0  # Normalize the pixel values
        images.append(curr_img)
        labels.append(label)
    return np.array(images), np.array(labels)

In [21]:
# Preprocess test images
test_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv')
test_images, test_labels = test_load_and_preprocess_images(test_df)

# Make predictions on the test set
predictions = model.predict(test_images)

# Assuming predictions are probabilities, you might want to convert them to class labels
predicted_labels = np.argmax(predictions, axis=1)

960/960 [==============================] - 9s 9ms/step


In [22]:
print(predicted_labels) # predicted classIDs
print(test_labels) # UniqueIDs

[1 7 4 ... 0 1 2]
[     1      9     10 ... 122871 122878 122880]


In [23]:
# Assuming test_labels contains unique IDs
data = {'uniqueID': test_labels, 'classID': predicted_labels}
result_df = pd.DataFrame(data)

# Save DataFrame to CSV
result_df.to_csv('/kaggle/working/predictions.csv', index=False)

In [24]:
result_df.head()

,uniqueID,classID
0,1,1
1,9,7
2,10,4
3,14,1
4,16,6


# Other Test Code

In [25]:
num_classes = len(y_train.unique())
print("num classes: " + str(num_classes))

num classes: 10


In [26]:
# Concatenate image features with additional information
combined = concatenate([x, additional_input])

# Dense layers for classification
x = Dense(64, activation='relu')(combined)
output = Dense(num_classes, activation='softmax')(x) # softmax activation for multi-class classification, num_classes is the number of classes

In [27]:
from tensorflow.keras.metrics import AUC

# Create the model
model = Model(inputs=[image_input, additional_input], outputs=output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
              metrics=[AUC()])  # Use AUC as the evaluation metric

In [28]:
print("datatype of images: " + str(images.dtype))
print("datatype of labels: " + str(labels) + "\n")
print("all_img_info: " + str(all_img_info))

datatype of images: float64
datatype of labels: [8 6 2 ... 6 4 1]

all_img_info: [[2 'occluded-specimens' 'L' 'a1a8b48e8cb142b3.jpg']
 [3 'microscope-slides' 'L' '79599db2ac9092b6.jpg']
 [4 'illustrations-color' 'BHL' 'c449696f2f0d0d92.jpg']
 ...
 [122876 'microscope-slides' 'L' '78f6868694a6669c.jpg']
 [122877 'live-plants' 'E' '5c6162948949510a.jpg']
 [122879 'biocultural-specimens' 'C' '58583a38131331e8.jpg']]


In [29]:
[print(i.shape, i.dtype) for i in model.inputs]
print("\n")
[print(o.shape, o.dtype) for o in model.outputs]
print("\n")
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 32, 32, 3) <dtype: 'float32'>
(None, 4) <dtype: 'float32'>


(None, 10) <dtype: 'float32'>


input_1 [(None, 32, 32, 3)] float32
conv2d (None, 32, 32, 3) float32
max_pooling2d (None, 30, 30, 32) float32
conv2d_1 (None, 15, 15, 32) float32
max_pooling2d_1 (None, 13, 13, 64) float32
conv2d_2 (None, 6, 6, 64) float32
flatten (None, 4, 4, 64) float32
dense (None, 1024) float32
input_2 [(None, 4)] float32
concatenate [(None, 1024), (None, 4)] float32
dense_2 (None, 1028) float32
dense_3 (None, 64) float32


[None, None, None, None, None, None, None, None, None, None, None, None]

In [30]:
images_np_arr = np.asarray(images)
images_np_arr.dtype
# maybe not

dtype('float64')

In [31]:
from tensorflow.keras.utils import to_categorical

# Train the model on the entire dataset
history = model.fit([images, all_img_info], labels, epochs=10, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# do whole processing again for test data

In [ ]:
# Load test dataset
test_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test.csv')

# Function to preprocess test images
def preprocess_test_images(df):
    images = []
    all_img_info = []
    labels = []
    for index, row in df.iterrows():
        img_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test/' + row['imageFile']
        label = row['classID']
        curr_img_info = row[X_cols].values
        curr_img = cv2.imread(img_path)
        curr_img = cv2.resize(curr_img, (32, 32))
        curr_img = curr_img / 255.0
        images.append(curr_img)
        all_img_info.append(curr_img_info)
        labels.append(label)
    return np.array(images), np.array(all_img_info), np.array(labels)

# Preprocess test images
test_images, test_all_img_info, test_labels = preprocess_test_images(test_df)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate([test_images, test_all_img_info], test_labels)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# import cv2
# import numpy as np
# import os

# # Define the directory containing your images
# image_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'

# # Function to load and preprocess images
# def load_images(image_dir):
#     images = []
#     labels = []
#     for label in os.listdir(image_dir):
#         label_path = os.path.join(image_dir, label)
#         for image_file in os.listdir(label_path):
#             image_path = os.path.join(label_path, image_file)
#             image = cv2.imread(image_path)
#             image = cv2.resize(image, (32, 32))  # Resize the image if necessary
#             image = image / 255.0  # Normalize the pixel values
#             images.append(image)
#             labels.append(int(label))  # Assuming each subdirectory is a class label
#     return np.array(images), np.array(labels)

# # Load and preprocess images
# images, labels = load_images(image_dir)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt

# # Load and prepare the CIFAR10 dataset
# # (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# # Normalize pixel values to be between 0 and 1
# train_images, test_images = train_images / 255.0, test_images / 255.0


# train_images

In [ ]:
# path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'
# #img_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train/0000000000010109.jpg'

# train_img_files = tf.data.Dataset.list_files(path)

# def load_images(path):
#     image = tf.io.read_file(path)
#     image = tf.io.decode_jpeg(image)
#     image = tf.image.convert_image_dtype(image, tf.float32)
#     image = tf.image.resize(image, (128, 128))

#     parts = tf.strings.split(path, os.path.sep)
#     bool_values = tf.equal(parts[-2], 'strokes')
#     indices = tf.cast(bool_values, tf.int32)
#     return image, indices

# ds = train_img_files.map(load_images).batch(1)

In [ ]:
# ds

In [ ]:
# ds.take(1)

In [ ]:
# for images, labels in ds.take(1):
#     for i in range(32):
#         ax = plt.subplot(6, 6, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(y_train[i])
#         plt.axis("off")

In [ ]:
# plt.imshow(ds[0].numpy().astype("uint8"))

In [ ]:
# # the data, split between train and test sets
# #(x_train, y_train), (x_test, y_test) = ds

# (x_train, y_train) = ds

# x_train

In [ ]:
# # Define the CNN architecture
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# # Add Dense layers on top
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10))

In [ ]:
# # Compile and train the model
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# history = model.fit(train_images, train_labels, epochs=10, 
#                     validation_data=(test_images, test_labels))

In [ ]:
# # Evaluate the model
# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
# print(test_acc)

In [ ]:
# fastai - cnn learner